## Generate predictions for base model and fine-tuned model

In [1]:
model_type = 'ft_model' # ft_model

In [2]:
! ls data

base_model_responses.jsonl
checkpoint-500-20240429T121103Z-001
checkpoint-500-20240429T121103Z-001.zip
ft_model_responses.jsonl
my-conll2003-dataset-test.jsonl
my-conll2003-dataset-test_sample.jsonl
my-conll2003-dataset-train.jsonl
my-conll2003-dataset-train_sample.jsonl
my-conll2003-dataset-validation.jsonl
my-conll2003-dataset-val_sample.jsonl


In [3]:
from datasets import load_dataset

# load data
dataset = json_datasets_reloaded = load_dataset("json", 
                            data_files="data/my-conll2003-dataset-test_sample.jsonl")


/home/areias/Documents/projects/mlops/mlflow10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_labels', 'sentence', 'entities'],
        num_rows: 100
    })
})

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

In [6]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [7]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.70s/it]


In [8]:
ls data/checkpoint-500-20240429T121103Z-001/checkpoint-500


adapter_config.json        optimizer.pt  rng_state.pth  trainer_state.json
adapter_model.safetensors  README.md     scheduler.pt   training_args.bin


In [9]:
from peft import PeftModel

if model_type == 'ft_model':
    ft_model = PeftModel.from_pretrained(model, "data/checkpoint-500-20240429T121103Z-001/checkpoint-500")

In [10]:
# Init an eval tokenizer that doesn't add padding or eos token
eval_tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    add_bos_token=True,
)


In [11]:
def eval_formatting_func(entry):
    # tokenizer adds bos token
    text = (f"[INST] You are an NLP expert tasked with Named Entity Extraction. "
        f"Identify entities of the type Person (PER), Organization (ORG), Location (LOC) and Miscellaneous (MISC) in the following sentence: '{entry['sentence']}'\n"
        "Your answer must be in the form of a dict {'PER':['person entity 1', 'person entity 2', '...'], 'ORG': [], 'LOC': [], 'MISC': []} \n"
        "Take care, your answer is only valid if it follows the correct format! [/INST]")

    return text


In [12]:
eval_prompt = eval_formatting_func(dataset['train'][10])
print(eval_prompt)

[INST] You are an NLP expert tasked with Named Entity Extraction. Identify entities of the type Person (PER), Organization (ORG), Location (LOC) and Miscellaneous (MISC) in the following sentence: 'LONDON 1996-12-07'
Your answer must be in the form of a dict {'PER':['person entity 1', 'person entity 2', '...'], 'ORG': [], 'LOC': [], 'MISC': []} 
Take care, your answer is only valid if it follows the correct format! [/INST]


In [13]:
print(eval_tokenizer.decode(eval_tokenizer(eval_prompt)['input_ids']))

<s> [INST] You are an NLP expert tasked with Named Entity Extraction. Identify entities of the type Person (PER), Organization (ORG), Location (LOC) and Miscellaneous (MISC) in the following sentence: 'LONDON 1996-12-07'
Your answer must be in the form of a dict {'PER':['person entity 1', 'person entity 2', '...'], 'ORG': [], 'LOC': [], 'MISC': []} 
Take care, your answer is only valid if it follows the correct format! [/INST]


In [14]:
"""#test
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    response  = eval_tokenizer.decode(model.generate(**model_input,
                    max_new_tokens=256, repetition_penalty=1.15,
                    pad_token_id=eval_tokenizer.eos_token_id)[0], skip_special_tokens=True)
    response = response.replace(eval_prompt, "")
    print(response)"""

 Based on the given input, there does not seem to be any identifiable PER, ORG, LOC or MISC entities present in the sentence 'LONDON 1996-12-07'. The sentence consists solely of a location (LONDON) and a date (1996-12-07). Therefore, the output should be:
{'PER': [], 'ORG': [], 'LOC': ['LONDON'], 'MISC': []}


In [16]:
import re
def parse_response(sentence):
    # Find the text between curly braces
    keys = ['PER', 'ORG', 'LOC', 'MISC']
    match = re.search(r'\{[^{}]+\}', response)
    # If a match is found, evaluate it as a dictionary using eval
    if match:
        entities_str = match.group(0)
        try:
            entities_dict = eval(entities_str)
            # check dictionary contains all keys, if not add it
            for key in keys:
                if key not in entities_dict:
                    entities_dict[key] = []
            return entities_dict
        # if you cant parse it as a dictionary return empty dictionary
        except:
            return {'PER':[], 'ORG':[], 'LOC':[], 'MISC':[]}
    else:
        # If no match is found, return an empty dictionary
        return {'PER':[], 'ORG':[], 'LOC':[], 'MISC':[]}

In [16]:
response

" Based on the given input, there does not seem to be any identifiable PER, ORG, LOC or MISC entities present in the sentence 'LONDON 1996-12-07'. The sentence consists solely of a location (LONDON) and a date (1996-12-07). Therefore, the output should be:\n{'PER': [], 'ORG': [], 'LOC': ['LONDON'], 'MISC': []}"

In [31]:
response_dict = parse_response(response)
response_dict

{'PER': [], 'ORG': [], 'LOC': ['LONDON'], 'MISC': []}

In [35]:
len(dataset['train'])

100

In [33]:
dataset['train'][25]

{'id': '2252',
 'tokens': ['Burmese',
  'students',
  'march',
  'out',
  'of',
  'campus',
  'again',
  '.'],
 'pos_tags': [21, 24, 41, 15, 15, 21, 30, 7],
 'chunk_tags': [11, 12, 21, 13, 13, 11, 3, 0],
 'ner_tags': [7, 0, 0, 0, 0, 0, 0, 0],
 'ner_labels': ['B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 'sentence': 'Burmese students march out of campus again .',
 'entities': {'LOC': [], 'MISC': ['Burmese'], 'ORG': [], 'PER': []}}

In [14]:
list(range(0,10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [14]:
from tqdm import tqdm
import jsonlines

# Define the file path for saving JSONL
if model_type == 'base_model':
    output_file = "data/base_model_responses.jsonl"
else:
    output_file = "data/ft_model_responses.jsonl"

output_file

'data/ft_model_responses.jsonl'

In [20]:

if model_type == "ft_model": # swith model in eval loop
    model = ft_model

for idx in tqdm(range(len(dataset['train']))):

    eval_prompt = eval_formatting_func(dataset['train'][idx])

    ground_truth = dataset['train'][idx]['entities']

    model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    model.eval()
    with torch.no_grad():
        response  = eval_tokenizer.decode(model.generate(**model_input,
                        max_new_tokens=256, repetition_penalty=1.15,
                        pad_token_id=eval_tokenizer.eos_token_id)[0], skip_special_tokens=True)
        response = response.replace(eval_prompt, "")

    response_dict = parse_response(response)

    response_data = {'id':  dataset['train'][idx]['id'],
                     'sentence': dataset['train'][idx]['sentence'],
                    'entities': dataset['train'][idx]['entities'],
                    model_type:#'base_model': 
                        {'response': response,
                        'response_dict': response_dict}
                        }
       
    # Save response data to JSONL file
    with jsonlines.open(output_file, mode='a') as writer:
        writer.write(response_data)


100%|██████████| 100/100 [03:08<00:00,  1.88s/it]


In [21]:
responses= []
with jsonlines.open(f'data/{model_type}_responses.jsonl', mode='r') as reader:
    for item in reader:
        responses.append(item)

In [22]:
len(responses)

100

In [23]:
responses

[{'id': '409',
  'sentence': 'Hartford 4 BOSTON 2',
  'entities': {'LOC': [],
   'MISC': [],
   'ORG': ['Hartford', 'BOSTON'],
   'PER': []},
  'ft_model': {'response': "\n{'LOC': ['BOSTON'], 'MISC': [], 'ORG': ['Hartford'], 'PER': []}",
   'response_dict': {'LOC': ['BOSTON'],
    'MISC': [],
    'ORG': ['Hartford'],
    'PER': []}}},
 {'id': '125',
  'sentence': 'S. Doull c subs ( M. Wasim ) b Waqar 1',
  'entities': {'LOC': [],
   'MISC': [],
   'ORG': [],
   'PER': ['S. Doull', 'M. Wasim', 'Waqar']},
  'ft_model': {'response': "\n{'LOC': [], 'MISC': ['Wasim'], 'ORG': [], 'PER': ['S. Doull', 'Waqar']}",
   'response_dict': {'LOC': [],
    'MISC': ['Wasim'],
    'ORG': [],
    'PER': ['S. Doull', 'Waqar']}}},
 {'id': '343',
  'sentence': 'Camilla Martin ( Denmark ) beat Wang Chen ( China ) 11-0 12-10',
  'entities': {'LOC': ['Denmark', 'China'],
   'MISC': [],
   'ORG': [],
   'PER': ['Camilla Martin', 'Wang Chen']},
  'ft_model': {'response': "\n{'LOC': ['Denmark', 'China'], 'MISC': 